In [ ]:
%load_ext autoreload
%autoreload 3

In [ ]:
import logging
import sys

import ferc1_transform_refactor as tfr
import pandas as pd

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter("%(message)s")
handler.setFormatter(formatter)
logger.handlers = [handler]

In [ ]:
dataset_params = tfr.TRANSFORM_PARAMS
dataset_params

In [ ]:
table_params = tfr.TableTransformParams.parse_obj(dataset_params["test_table_one"])
table_params

In [ ]:
test_input = tfr.TEST_INPUT_DF
test_input

In [ ]:
tttfr_one = tfr.TestTableTransformerOne()
actual = tttfr_one.transform(df=test_input)
actual

In [ ]:
expected = tfr.EXPECTED_OUTPUT_DFS["test_table_one"]
expected

In [ ]:
pd.testing.assert_frame_equal(
    actual.sort_index(axis="columns"),
    expected.sort_index(axis="columns")
)